In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install fasttext

In [ ]:
seed_value = 777

In [ ]:
import fasttext

import pandas as pd
import numpy as np
import random
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight


from sklearn.metrics import accuracy_score

from lightgbm import LGBMClassifier

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

import joblib

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model

tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
train_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_train.csv')
test_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_test.csv')
label_sample_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_label_sample.csv')

In [ ]:
print(label_sample_org.shape)
print(train_org.shape)
print(test_org.shape)

(4327, 2)
(9322, 13)
(37671, 12)


In [ ]:
train_change_msg= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/train_map_3.csv')
test_change_msg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_map_3.csv')

In [ ]:
print(train_change_msg.shape)
print(test_change_msg.shape)

(9322, 13)
(37671, 12)



# **transformer**

In [ ]:
# transformer 모델의 전처리를 완료한 데이터 파일

train_dup = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/train_site.csv')
test_dup = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_site.csv')

In [ ]:
train_msg_tf = train_dup.copy()
train_msg_tf.head()

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail
1,14771766.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
2,14777089.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
3,14790052.0,POWER-SUPPLY-UNIT-FAIL,PowerFail
4,14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail


In [ ]:
test_msg_tf = test_dup.copy()
test_msg_tf.head()

,ticketno,alarmmsg_original
0,15238899.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
1,15712444.0,POWER-SUPPLY-UNIT-FAIL
2,15723187.0,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
3,15737103.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
4,15737132.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"


In [ ]:
def unpack_list(lst):
    """
    list를 str로 변환하는 함수

    Args:
        lst (list): str로 변환하고 싶은 list

    Returns:
        str로 변환된 list의 요소들을 ','로 연결시킨 문자열
    """

    return ','.join(str(x) for x in lst)

In [ ]:
def replace_root_cause_type(row):
    if 'PowerFail' in row:
        return 'PowerFail'
    elif 'UnitFail' in row:
        return 'UnitFail'
    elif 'LinkCut' in row:
        return 'LinkCut'
    return row


## **tokenized**

In [ ]:
train_msg_tf['alarmmsg_original'] = train_msg_tf['alarmmsg_original'].str.replace('-', ' ')
train_msg_tf['alarmmsg_original'] = train_msg_tf['alarmmsg_original'].apply(lambda x: x.split(','))
train_msg_tf['alarmmsg_original'] = train_msg_tf['alarmmsg_original'].apply(lambda x: ' '.join(x))
train_msg_tf.head()

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC...,PowerFail
1,14771766.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL,PowerFail
2,14777089.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL,PowerFail
3,14790052.0,POWER SUPPLY UNIT FAIL,PowerFail
4,14879922.0,ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC...,PowerFail


In [ ]:
# 1. Tokenization

texts = train_msg_tf['alarmmsg_original'].tolist()

tokenized_texts = [text.split() for text in texts]

In [ ]:
# 단어를 고유한 정수로 매핑하는 사전 생성
# 중복된 단어를 처리하기 위해 Counter를 사용해서 정수 index 할당
from collections import Counter

word_counts = Counter(word for text in tokenized_texts for word in text)
word_to_index = {"[PAD]": 0, "[UNK]": 1}
for word in word_counts:
    if word not in word_to_index:
        word_to_index[word] = len(word_to_index)

In [ ]:
# 단어를 정수로 매핑된 값으로 변환

mapped_texts = [[word_to_index[word] for word in text] for text in tokenized_texts]

In [ ]:
len(mapped_texts)

1114

In [ ]:
# test length 확인 후, 더 큰 값으로 변경 필요
# max_length = max(len(text) for text in mapped_texts)

max_length = 41

padded_texts = [text + [0] * (max_length - len(text)) for text in mapped_texts]

In [ ]:
max_length

41

In [ ]:
len(padded_texts)

1114

In [ ]:
# 5개의 결과 확인
for i, text in enumerate(padded_texts[:5]):
    print('Original Text: {}'.format(texts[i]))
    print('Mapped Text: {}'.format(text))
    print('-' * 30)

Original Text: ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 2, 5, 6, 7, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: POWER SUPPLY UNIT FAIL
Mapped Text: [8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Ma

In [ ]:
# 단어와 정수 index가 매핑된 dict 생성

text_dict = {texts[i]: text for i, text in enumerate(padded_texts)}

## **test_tokenized**

In [ ]:
test_msg_tf = test_dup.copy()

In [ ]:
test_msg_tf['alarmmsg_original'] = test_msg_tf['alarmmsg_original'].str.replace('-', ' ')
test_msg_tf['alarmmsg_original'] = test_msg_tf['alarmmsg_original'].apply(lambda x: x.split(','))
test_msg_tf['alarmmsg_original'] = test_msg_tf['alarmmsg_original'].apply(lambda x: ' '.join(x))
test_msg_tf.head()

,ticketno,alarmmsg_original
0,15238899.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
1,15712444.0,POWER SUPPLY UNIT FAIL
2,15723187.0,ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT ...
3,15737103.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
4,15737132.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL


In [ ]:
texts_test = test_msg_tf['alarmmsg_original'].tolist()

tokenized_texts_test = [text.split() for text in texts_test]

In [ ]:
# 새로운 단어를 word_to_index 매핑에 추가

for sentence_tokens in tokenized_texts_test:
    for word in sentence_tokens:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

print('Updated word_to_index: \n', word_to_index)

Updated word_to_index: 
 {'[PAD]': 0, '[UNK]': 1, 'ETHERNET': 2, 'LINK': 3, 'FAIL': 4, 'NO': 5, 'RECEIVE': 6, 'TRAFFIC': 7, 'POWER': 8, 'SUPPLY': 9, 'UNIT': 10, '48V': 11, 'FAN': 12, 'OPTICAL': 13, 'LOSS': 14, 'OF': 15, 'SIGNAL': 16, 'REMOVE': 17, 'NON': 18, 'VOLATILE': 19, 'RANDOM': 20, 'ACCESS': 21, 'MEMORY': 22, 'BATTERY': 23, 'LOW': 24, 'DATA': 25, 'COMMUNICATE': 26, 'CHANNEL': 27, 'TRANSMISSION': 28, 'PROTECTION': 29, 'SWITCHED': 30, 'LABEL': 31, 'PATH': 32, 'CONNECT': 33, 'AUTO': 34, 'NEGOTIATION': 35, 'MISMATCH': 36, 'BOOTING': 37, 'OPERATING': 38, 'SYSTEM': 39, 'RESTARTED': 40, 'PORT': 41, 'SHUTDOWN': 42, 'TIME': 43, 'DIVISION': 44, 'MULTIPLEX': 45, 'PSEUDOWIRE': 46, 'PACKET': 47, 'INTEGRATED': 48, 'CONTROL': 49, 'TRACK': 50, 'ERROR': 51, 'REMOTE': 52, 'QUALITY': 53, 'MANUAL': 54, 'ALARM': 55, 'MAINTENANCE': 56, 'END': 57, 'POINT': 58, 'HIGH': 59, 'CLIENT': 60, 'DROP': 61, '1': 62, 'DAY': 63, 'EMULATION': 64, 'SERVER': 65, '15': 66, 'MINUTE': 67, 'ENVIRONMENT': 68, 'INPUT': 69,

In [ ]:
mapped_texts_test = []
for sentence_tokens in tokenized_texts_test:
    sentence_indices = [word_to_index[word] if word in word_to_index else word_to_index['[UNK]'] for word in sentence_tokens]
    mapped_texts_test.append(sentence_indices)

In [ ]:
len(mapped_texts_test)

4327

In [ ]:
# max_length_test = 41
max_length_test = max(len(text) for text in mapped_texts_test)

padded_texts_test = [text + [0] * (max_length_test - len(text)) for text in mapped_texts_test]

In [ ]:
max_length_test

41

In [ ]:
len(padded_texts_test)

4327

In [ ]:
for i, text in enumerate(padded_texts_test[:5]):
    print('Original Text: {}'.format(texts_test[i]))
    print('Mapped Text: {}'.format(text))
    print('-' * 30)

Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: POWER SUPPLY UNIT FAIL
Mapped Text: [8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Mapped Text: [2, 5, 6, 7, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0

# **FastText**

In [ ]:
# FastText 모델의 전처리를 완료한 데이터 파일

train_FT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/train_FT.csv')
test_FT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_FT.csv')

In [ ]:
# FastText로 vector화 된 data

test_lgbm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_lgbm.csv')

In [ ]:
class_labels = ['LinkCut', 'PowerFail', 'UnitFail']
class_weights = compute_class_weight('balanced', classes = class_labels, y = train_FT['root_cause_type'])

class_weight_dict = {class_labels[i]: class_weights[i] for i in range(len(class_labels))}

# **fasttext**

In [ ]:
# fasttext를 위한 중복 제거를 완료한 data

train_ft = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/train_last_asc2_site.csv')
test_ft = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_last_asc2_site.csv')

In [ ]:
print(train_ft.shape)
print(test_ft.shape)

(1114, 4)
(4327, 3)


In [ ]:
msg_train_ft = train_ft['alarmmsg_original']
print(len(msg_train_ft))

1114


0       POWER-SUPPLY-UNIT-FAIL,ETHERNET-NO-RECEIVE-TRA...
1               POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL
2               POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL
3                                  POWER-SUPPLY-UNIT-FAIL
4       POWER-SUPPLY-UNIT-FAIL,ETHERNET-NO-RECEIVE-TRA...
5       POWER-SUPPLY-UNIT-FAIL,ETHERNET-NO-RECEIVE-TRA...
6               POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL
7                              FAN-FAIL,48V-FAIL,48V-FAIL
8                   OPTICAL-REMOVE,OPTICAL-LOSS-OF-SIGNAL
9                  NON-VOLATILE-RANDOM-ACCESS-MEMORY-FAIL
10                                 POWER-SUPPLY-UNIT-FAIL
11                                 POWER-SUPPLY-UNIT-FAIL
12                                 POWER-SUPPLY-UNIT-FAIL
13              POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL
14              POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL
15                                            BATTERY-LOW
16                                 POWER-SUPPLY-UNIT-FAIL
17            

In [ ]:
# 문장에서 단어를 분리할 때 사용한 ','를 공백으로 대체
msg_train_ft = [x.replace(',', ' ') for x in msg_train_ft]
len(msg_train_ft)

1114

In [ ]:
msg_test_ft = test_ft['alarmmsg_original']
print(len(msg_test_ft))

4327


In [ ]:
# 문장에서 단어를 분리할 때 사용한 ','를 공백으로 대체
msg_test_ft = [x.replace(',', ' ') for x in msg_test_ft]
len(msg_test_ft)

4327

# **train**

In [ ]:
train_ft.head()

,alarmlevel,alarmmsg_original,site,root_cause_type
0,"4,5,7","POWER-SUPPLY-UNIT-FAIL,ETHERNET-NO-RECEIVE-TRA...","ABNE,ABNE,ABNE",PowerFail
1,"4,7","POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL","ABLB,ABLB",PowerFail
2,"4,7","POWER-SUPPLY-UNIT-FAIL,ETHERNET-LINK-FAIL","ABLZ,ABLZ",PowerFail
3,4,POWER-SUPPLY-UNIT-FAIL,ACRX,PowerFail
4,"4,5,7","POWER-SUPPLY-UNIT-FAIL,ETHERNET-NO-RECEIVE-TRA...","ABNE,ABNE,ABNE",PowerFail


In [ ]:
# '-'와 ',' 제거 함수 정의
def remove_chars(text):
    text = text.replace('-', ' ').replace(',', ' ')
    return text

# apply 함수를 사용하여 컬럼의 값에 함수 적용
train_ft['alarmmsg_original'] = train_ft['alarmmsg_original'].apply(remove_chars)

In [ ]:
# fasttext에 input 데이터의 형식을 지키기 위해 변환 필요
# 레이블 형식으로 변환하는 함수 정의
def convert_to_label(row):
    return f'__label__{row}'

# 'root_cause_type' 컬럼 값 변환
train_ft['root_cause_type'] = train_ft['root_cause_type'].apply(convert_to_label)

In [ ]:
col = ['root_cause_type', 'alarmmsg_original']
input_train_ft = train_ft[col]      # input train data
input_train_ft.head()

,root_cause_type,alarmmsg_original
0,__label__PowerFail,POWER SUPPLY UNIT FAIL ETHERNET NO RECEIVE TRA...
1,__label__PowerFail,POWER SUPPLY UNIT FAIL ETHERNET LINK FAIL
2,__label__PowerFail,POWER SUPPLY UNIT FAIL ETHERNET LINK FAIL
3,__label__PowerFail,POWER SUPPLY UNIT FAIL
4,__label__PowerFail,POWER SUPPLY UNIT FAIL ETHERNET NO RECEIVE TRA...


In [ ]:
# txt 파일로 저장 (fasttex에 넣으려면 txt 형식 필요)

output_file_path = 'input_train_last_asc2.txt'
input_train_ft.to_csv(output_file_path, sep = '\t', index = False, header = False)

# **test**

In [ ]:
# '-'와 ',' 제거 함수 정의

def remove_chars(text):
    text = text.replace('-', ' ').replace(',', ' ')
    return text

# apply 함수를 사용하여 컬럼의 값에 함수 적용
test_ft['alarmmsg_original'] = test_ft['alarmmsg_original'].apply(remove_chars)

In [ ]:
col = ['alarmmsg_original']
output_test_ft = test_ft[col]
output_test_ft.head()

,alarmmsg_original
0,POWER SUPPLY UNIT FAIL ETHERNET LINK FAIL
1,POWER SUPPLY UNIT FAIL
2,POWER SUPPLY UNIT FAIL ETHERNET NO RECEIVE TRA...
3,POWER SUPPLY UNIT FAIL ETHERNET LINK FAIL
4,POWER SUPPLY UNIT FAIL ETHERNET LINK FAIL


In [ ]:
# txt 파일로 저장
output_file_path = 'output_test_last_asc2.txt'
output_test_ft.to_csv(output_file_path, sep = '\t', index = False, header = False)

# **Voting**

In [ ]:
# transformer model
model_tf = load_model('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/model/transformer_model.h5')

# FastText로 alarmmsg를 처리한 lightgbm model
model_lgbm = joblib.load('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/model/lightgbm_model.pkl')

# fasttext model
model_ft = fasttext.load_model('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/model/fasttext_model.bin')

In [ ]:
# 단어 인덱싱 된 data
test_input_tf = np.array(padded_texts_test)

# 벡터화 된 data
test_input_lgbm = np.array(test_lgbm['alarmmsg_original'])
test_input_lgbm = np.array([np.fromstring(s[1:-1], sep = ' ') for s in test_input_lgbm])

# 단어 data
test_input_ft = np.array(output_test_ft['alarmmsg_original'])

In [ ]:
predict_tf = model_tf.predict(test_input_tf)

136/136 [==============================] - 12s 87ms/step


In [ ]:
predict_lgbm = model_lgbm.predict_proba(test_input_lgbm)

In [ ]:
# 예측 수행 및 결과 저장
predictions_ft = []
for line in test_input_ft:
    line = line.strip()  # 줄바꿈 문자 제거
    pred_label = model_ft.predict((line), k = 3)
    predictions_ft.append(pred_label)

In [ ]:
predict_ft = []

for labels, probabilities in predictions_ft:
    label_order = ['__label__LinkCut', '__label__PowerFail', '__label__UnitFail']
    sorted_indices = [labels.index(label) for label in label_order]  # 레이블 순서에 따른 인덱스 배열
    sorted_labels = np.array(labels)[sorted_indices]  # 레이블 순서대로 정렬된 레이블 배열
    sorted_probs = probabilities[sorted_indices]  # 레이블 순서대로 정렬된 확률 배열
    numbers_only = sorted_probs.tolist()  # 확률 배열에서 숫자만 추출하여 리스트로 변환
    predict_ft.append(numbers_only)

predict_ft = np.array(predict_ft)

In [ ]:
# 각 모델들의 predict_proba 를 이용해서 soft voting

average_prediction = (predict_tf + predict_lgbm + predict_ft) / 3

In [ ]:
voting_result = np.argmax(average_prediction, axis = 1)

predicted_labels = []
for idx in voting_result:
    if idx == 0:
        predicted_labels.append('LinkCut')
    elif idx == 1:
        predicted_labels.append('PowerFail')
    else:
        predicted_labels.append('UnitFail')

In [ ]:
label_sample = label_sample_org.copy()
label_sample['root_cause_type'] = predicted_labels

In [ ]:
label_sample['root_cause_type'].value_counts()/len(label_sample['root_cause_type'])

LinkCut      0.652877
PowerFail    0.314074
UnitFail     0.033048
Name: root_cause_type, dtype: float64